In [11]:
import httpx
import time

def wait_for_vllm_ready(url="http://localhost:8000/v1/models", timeout=300):
    print("⏳ Waiting for vLLM API to be ready...")
    start_time = time.time()
    while True:
        try:
            response = httpx.get(url, timeout=2)
            if response.status_code == 200:
                print("✅ vLLM is ready!")
                break
        except Exception as e:
            if time.time() - start_time > timeout:
                raise TimeoutError("❌ Timed out waiting for vLLM.") from e
            time.sleep(5)

wait_for_vllm_ready()


⏳ Waiting for vLLM API to be ready...


TimeoutError: ❌ Timed out waiting for vLLM.

In [13]:
!lsof -i :8000

/bin/bash: lsof: command not found


In [8]:
from openai import OpenAI
import json
import ast
from pathlib import Path
import traceback
from tqdm.notebook import tqdm

# initial VLLM
client = OpenAI(
    base_url='http://localhost:8000/v1',
    api_key='EMPTY', 
)

In [9]:
# Define api sturcture
def openai_api_predict(model, query):
    response = client.chat.completions.create(
        model=model,
        messages=[
            {'role': 'system', 'content': 'You are a helpful assistant.'},
            {'role': 'user', 'co*ntent': query}
        ],
        stream=False
    )
    return response.choices[0].message.content

def read_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return json.load(file)

def write_json(data, file_path, indent=4):
    Path(file_path).parent.mkdir(parents=True, exist_ok=True)
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False, indent=indent)


In [5]:
# Setting model and path
model_name = 'Qwen/Qwen2.5-1.5B-Instruct'
test_name = 'ner'

data_path = Path('/scratch/project_2005072/keshu/cascade-camp-hands-on/data')
data_file = data_path / 'topres19th/HIPE-prep.json'
pred_dir = f'output/{model_name}_{test_name}.json'

# read data
qas = read_json(data_file)
print(f"Loaded {len(qas)} entries.")

Loaded 309 entries.


In [10]:
# Running inference

preds = []

for qa in tqdm(qas[:20]):  # manage to test a small subset, clear to run all
    text = qa['text']

    query = f'''
    This is a named entity recognition task, which consists of two steps:
    1) First, identify all entity mentions in the text.
    2) Then classify each mention into one of the following categories:
    ["LOC", "STREET", "BUILDING"].

    Given the following text:
    {text}

    Output format: {{"LOC": [...], "STREET": [...], "BUILDING": [...]}}
    Do not provide any explanation.
    '''

    try:
        start = time.time()
        answer = openai_api_predict(model_name, query)
        duration = time.time() - start
        print(f"Model output: {answer}")
        print(f"⏱ Time: {duration:.2f}s\n")

        parsed = ast.literal_eval(answer)
        parsed = json.loads(json.dumps(parsed))  # make sure it returns JSON

    except Exception as e:
        print(f"⚠️ Error: {e}")
        traceback.print_exc()
        parsed = {"LOC": [], "STREET": [], "BUILDING": []}

    preds.append({
        "text": text,
        "preds": parsed
    })


  0%|          | 0/20 [00:00<?, ?it/s]

⚠️ Error: Connection error.


Traceback (most recent call last):
  File "/usr/local/lib/python3.12/site-packages/httpx/_transports/default.py", line 101, in map_httpcore_exceptions
    yield
  File "/usr/local/lib/python3.12/site-packages/httpx/_transports/default.py", line 250, in handle_request
    resp = self._pool.handle_request(req)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/site-packages/httpcore/_sync/connection_pool.py", line 256, in handle_request
    raise exc from None
  File "/usr/local/lib/python3.12/site-packages/httpcore/_sync/connection_pool.py", line 236, in handle_request
    response = connection.handle_request(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/site-packages/httpcore/_sync/connection.py", line 101, in handle_request
    raise exc
  File "/usr/local/lib/python3.12/site-packages/httpcore/_sync/connection.py", line 78, in handle_request
    stream = self._connect(request)
             ^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/loca

⚠️ Error: Connection error.


Traceback (most recent call last):
  File "/usr/local/lib/python3.12/site-packages/httpx/_transports/default.py", line 101, in map_httpcore_exceptions
    yield
  File "/usr/local/lib/python3.12/site-packages/httpx/_transports/default.py", line 250, in handle_request
    resp = self._pool.handle_request(req)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/site-packages/httpcore/_sync/connection_pool.py", line 256, in handle_request
    raise exc from None
  File "/usr/local/lib/python3.12/site-packages/httpcore/_sync/connection_pool.py", line 236, in handle_request
    response = connection.handle_request(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/site-packages/httpcore/_sync/connection.py", line 101, in handle_request
    raise exc
  File "/usr/local/lib/python3.12/site-packages/httpcore/_sync/connection.py", line 78, in handle_request
    stream = self._connect(request)
             ^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/loca

...

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/site-packages/httpx/_transports/default.py", line 101, in map_httpcore_exceptions
    yield
  File "/usr/local/lib/python3.12/site-packages/httpx/_transports/default.py", line 250, in handle_request
    resp = self._pool.handle_request(req)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/site-packages/httpcore/_sync/connection_pool.py", line 256, in handle_request
    raise exc from None
  File "/usr/local/lib/python3.12/site-packages/httpcore/_sync/connection_pool.py", line 236, in handle_request
    response = connection.handle_request(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/site-packages/httpcore/_sync/connection.py", line 101, in handle_request
    raise exc
  File "/usr/local/lib/python3.12/site-packages/httpcore/_sync/connection.py", line 78, in handle_request
    stream = self._connect(request)
             ^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/loca

⚠️ Error: Connection error.


Traceback (most recent call last):
  File "/usr/local/lib/python3.12/site-packages/httpx/_transports/default.py", line 101, in map_httpcore_exceptions
    yield
  File "/usr/local/lib/python3.12/site-packages/httpx/_transports/default.py", line 250, in handle_request
    resp = self._pool.handle_request(req)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/site-packages/httpcore/_sync/connection_pool.py", line 256, in handle_request
    raise exc from None
  File "/usr/local/lib/python3.12/site-packages/httpcore/_sync/connection_pool.py", line 236, in handle_request
    response = connection.handle_request(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/site-packages/httpcore/_sync/connection.py", line 101, in handle_request
    raise exc
  File "/usr/local/lib/python3.12/site-packages/httpcore/_sync/connection.py", line 78, in handle_request
    stream = self._connect(request)
             ^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/loca

⚠️ Error: Connection error.


Traceback (most recent call last):
  File "/usr/local/lib/python3.12/site-packages/httpx/_transports/default.py", line 101, in map_httpcore_exceptions
    yield
  File "/usr/local/lib/python3.12/site-packages/httpx/_transports/default.py", line 250, in handle_request
    resp = self._pool.handle_request(req)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/site-packages/httpcore/_sync/connection_pool.py", line 256, in handle_request
    raise exc from None
  File "/usr/local/lib/python3.12/site-packages/httpcore/_sync/connection_pool.py", line 236, in handle_request
    response = connection.handle_request(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/site-packages/httpcore/_sync/connection.py", line 101, in handle_request
    raise exc
  File "/usr/local/lib/python3.12/site-packages/httpcore/_sync/connection.py", line 78, in handle_request
    stream = self._connect(request)
             ^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/loca

In [ ]:
# Save results
write_json(preds, pred_dir)
print(f"✅ Saved {len(preds)} results to: {pred_dir}")